In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/생성 AI 모델링/data/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

- 데이터 불러오기

In [ ]:
train = pd.read_csv(f'{DATA_PATH}content_shuffled_df.csv')

# 변경사항 변경사항 +
-

# 1) 맞춤법 검사 및 변형



## Pyhanspell (맞춤법 검사기)
- https://github.com/ssut/py-hanspell
- 네이버 맞춤법 검사기와 동일. 띄어쓰기 검사 포함이므로 이거 사용하면 될 듯

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git

fatal: destination path 'py-hanspell' already exists and is not an empty directory.


In [ ]:
!cd py-hanspell


In [ ]:
# !python setup.py install


In [ ]:
!pip install py-hanspell/

Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4811 sha256=864ea9e90932e838cf28313b0d35b13389b6aec90e55d4dec093b96bbf2ab3d8
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell
  Attempting uninstall: py-hanspell
    Found existing installation: py-hanspell 1.1
    Uninstalling py-hanspell-1.1:
      Successfully uninstalled py-hanspell-1.1


- 함수부분!

In [ ]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
correct_spell('식사는 맛있게 하셨나요? 혜화 대학로 SNS피자랭킹맛집피자보이시나 대학로점을 믿고주...') #띄어쓰기는 잘됨


'식사는 맛있게 하셨나요? 혜화 대학로 SNS 피자 랭킹 맛집 피자 보이 시나 대학로점을 믿고 주...'

In [ ]:
correct_spell( '마시써요 최근와서 잘 먹기 시작한거 맞아요') # 일반적인 오류는 잡음


'맛있어요 최근 와서 잘 먹기 시작한 거 맞아요'

In [ ]:
correct_spell('리뷰작성 해주셔서정말 감사합니다!!!!르ㅡㅇㄴ')   # 초성만 있는 경우 ㅋㅋㅋㅋ, ㅎㅎㅎ 등 제외 없애야 할듯

'리뷰 작성해 주셔서 정말 감사합니다!!!!르ㅡㅇㄴ'

In [ ]:
correct_spell('다음에는 막창 구이 시켜먹어볼랴구요!') #


'다음에는 막창 구이 시켜 먹어볼 야구요!'

In [ ]:
correct_spell('요거트소스 하나 덜 왓는데요 ??????? 담에 하나 더 넣어주게용 치자는 맛잇어요') #용 처리

'요구르트 소스 하나 덜 왔는데요 ??????? 담에 하나 더 넣어주게요 치자는 맛있어요'

In [ ]:
correct_spell('마시씁니당당당')

'마시 씁니다 당당'

- hanspell에는 내장된 사용자 규칙 지정 기능이 없어서 수동으로 추가해야 할듯

In [ ]:

def custom_correction(text):
    corrected = spell_checker.check(text).checked
    # 특정 단어 변경 (예: '__'를 '말되도록'으로 변경)
    custom_corrected = corrected.replace('말되게', '말되도록')
    return custom_corrected


## 요기요 리뷰 적용
- input 및 output 모든 전처리 에서 사용하면 될듯
- input pyhanspell -> 사전학습모델 학습 -> 결과 -> 결과 pyhanspell -> reply  ?

In [ ]:
review_list

['오늘도맛있게해주셔서감사합니다',
 '너무 맛있어요 감사합니다 ',
 '먹다가 정신이 없어서 2조각 순삭하고 찍었네요 ',
 '맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요',
 '주말저녁 맛있게먹었어요',
 '좋았습니다 김치 더 주시면 감사하겠습니다',
 '진짜 너무 맛있어요 여기에서만 시켜용',
 '첫 주문이였는데 기대 이상입니다! 덕분에 오랜만애 기분좋은 시간 보냈습니다오자마자 허겁지겁먹어서 사진은 깜빡했네요 종종 또 찾아 주문 하겠습니다!!',
 '다리살이라 맛잇어욥양념도 자랑할만',
 '맛있게 먹었습니다. 양이 정말 많네요 다음에 또 시킬 거 같아요',
 '이틀 연속 시킬정도로 존맛',
 '저번 정식도 맛있었는데 가자미 미역국도 맛있네요비린내 걱정을 했는데 전혀 안나고 시원 담백합니다몸 안좋거나 해장할때 너무 좋을듯',
 '토핑도 풍부하고 맛있게 잘먹었습니다',
 '너무 맛있었고요 추가주문한게 빠졌다고 연락드렸더니 서비스와 함께 바로 보내주셨어요자주 시켜먹게 될 것 같아요',
 '와플은  바삭바삭맛있어요떡볶이도 괜찮긴한데...국물과떡이 조금 따로 노는 기분이에요..  떡이 그래서 흰색에 가까웠어요  맛 자체는 괜찮은 편입니다많이파세요',
 '맛있게 잘 먹었습니다',
 '맛있습니다 양도 괜찮습니다',
 '피자위에 콜라를 올려두셨는지모르겠고한데젖어있고 피자가 좀 찌그러져있고치즈가 이미 식어서 굳어있네요좀 아쉽고 한데피자는 치즈가 늘어나는맛에 먹는건데다음엔 따듯한걸먹길...',
 '리뷰가 늦었네요 너무 맛있게 먹었습니다!!진짜 존맛!!',
 '맛있어요......',
 '육회비빔밥은 여기서만 시킵니다 진짜 너무맛있어요 집근처 지점말고 본점에서만 계속주문합니다 번창하세요',
 '피자는 피보시 배달도 빠르고 서비스도 짱이고 뭣보다 맛있음',
 '그릇이 특이해요 사이드메뉴가 맛있어요 새우링 체고',
 '리뷰늦었지만  진짜 맛있게잘먹었습니다  요즘 피자여기서밖에안먹어요',
 '처음으로 훈제치킨 조합으로 먹었는데 역시 실망없는 맛이였습니다',
 '꽁치도 맛

In [ ]:
review_list = train['review'].tolist()

len(review_list)

15200

In [ ]:
crr_review_list = []
for review in review_list[:100]:
    corrected_review = correct_spell(review)
    crr_review_list.append(corrected_review)

In [ ]:
data = {'original': review_list[:100], 'corrected': crr_review_list}
df = pd.DataFrame(data)
df

#존만,

,original,corrected
0,오늘도맛있게해주셔서감사합니다,오늘도 맛있게 해주셔서 감사합니다
1,너무 맛있어요 감사합니다,너무 맛있어요 감사합니다
2,먹다가 정신이 없어서 2조각 순삭하고 찍었네요,먹다가 정신이 없어서 2조각 순삭 하고 찍었네요
3,맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요,맛있어요. 기존에 시켜 먹던데 버리고. 여기서만 시키게요
4,주말저녁 맛있게먹었어요,주말 저녁 맛있게 먹었어요
...,...,...
95,밑반찬도맛있고 오징어도 많이들었습니다 좋아요,밑반찬도 맛있고 오징어도 많이 들었습니다 좋아요
96,너무 맛있게 잘먹었어요 하 감동스러운 맛입니다,너무 맛있게 잘 먹었어요 하 감동스러운 맛입니다
97,맛있고 양이 많아요,맛있고 양이 많아요
98,잘 먹었습니다 술안주로도 좋아요,잘 먹었습니다 술안주로도 좋아요


## 답변 생성에 적용
- 맞춤법 검사 이외에 :
- 잘리는 문장 제거 or 가공해서 온전한 문장만 출력
- 고유명사, 의성어, 예외 표현들 제거

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024,
                                          padding_side='left')

# 2) 리뷰 긍부정 평가 모델
- MLOpsEngineer/review-rate-prediction (hugging face)

## 사전학습 모델 사용 - Review Rate Prediction

In [ ]:
model = 'MLOpsEngineer/review-rate-prediction'

In [ ]:
train

,review,owner_reply
0,오늘도맛있게해주셔서감사합니다,"m0님,안녕하세요 시간내어 리뷰작성해주셔서 정말 감사합니다항상 고객님들 주문 하나하..."
1,너무 맛있어요 감사합니다,저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다계속해서 더 많은 기대 부탁드...
2,먹다가 정신이 없어서 2조각 순삭하고 찍었네요,식사는 맛있게 하셨나요? 혜화 대학로 SNS피자랭킹맛집피자보이시나 대학로점을 믿고주...
3,맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요,만족해주신 그 맛! 절대 변하지 않고 앞으로도 쭉 만족하실 수 있도록 최선을 다하겠...
4,주말저녁 맛있게먹었어요,안녕하세요 고객님 D번거로우실텐데 리뷰 작성해 주셔서 감사합니다!앞으로도 맛있게 ...
...,...,...
15195,맛도 너무 좋고 양도 넉넉하고 시간은 겁나 빠르게 맛있습니다. 재주문 나중...,맛있게 드셔주셔서 감사드립니다!!맛도 좋고 양도 좋고 가성비좋고3박자가 다 어우러진...
15196,혼자먹기 많아용 배달은 20분만에 왔어요 잘먹었습니다,황궁 찾아주셔서 넘넘 감사합니다세스코사용으로 청결하고 빠른배달과 친절한서비스로찾아뵙...
15197,항상 맛있게 먹고 있습니다.,안녕하세요.삽교원조두리곱창입니다.저희 음식이 입맛에 맞으셨다니 정말 다행이고 뿌듯합...
15198,맛있어서 먹다가 중간에 찍었어요 3인이서 폭풍흡입중 맛있네요,고돼지 송파점입니다 별점만점 그리고 따뜻한 리뷰 작성해주셔서 감사합니다.위생과 청결...


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.3 MB/s eta 0:00:00


### 모델 구조 확인
- 원래 식당 별점을 평가하는 모델임. 리뷰를 입력받으면, 어떤 라벨에 해당하는지 알려줌
- Label_0 = 1점, Label_1 = 2점, Label_2 = 3점, Label_3 = 4점, Label_4 = 5점


In [ ]:
from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name)


In [ ]:
classifier = pipeline("sentiment-analysis", model=model_name)


In [ ]:
classifier(review_text)[0]

{'label': 'LABEL_3', 'score': 0.5232095122337341}

In [ ]:
# 분류할 리뷰 텍스트
review_text = "맛있어요!"

# 모델 추론
result = classifier(review_text)[0]

# 결과 해석
label = result['label']
score = result['score']

if label in ['LABEL_0','LABEL_1'] :   # 어떤 기준으로 label 1, 2, 3을 잡을지 확인 필요. 대략은 0,1만 부정으로 하는게 좋아보임
    sentiment = '부정'
else:
    sentiment = '긍정'

# 출력
print(f"리뷰의 감정: {sentiment}")
print(f"긍부정 지수: {score}")


리뷰의 감정: 긍정
긍부정 지수: 0.5232095122337341


### 모델 사용

In [ ]:
import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews, model_name):
    # 모델 불러오기
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in reviews:
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = '부정'
        else:
            sentiment = '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    df = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return df

In [ ]:
# 분류할 리뷰 리스트
reviews = [
    "이 영화 정말 좋았습니다. 감동적인 내용과 연기가 인상 깊었어요.",
    "아쉬운 영화였습니다. 전반적으로 기대에 못 미쳤습니다.",
    "이 책 너무 재미있어요! 추천합니다.",
    "별로인 영화. 보지 않는 것을 추천합니다."
]

model_name = "MLOpsEngineer/review-rate-prediction"


In [ ]:
# 리뷰 분석 및 데이터프레임 반환
result_df = analyze_reviews(reviews, model_name)
pd.DataFrame(result_df)

,review,sentiment,score
0,이 영화 정말 좋았습니다. 감동적인 내용과 연기가 인상 깊었어요.,긍정,0.815933
1,아쉬운 영화였습니다. 전반적으로 기대에 못 미쳤습니다.,부정,0.916873
2,이 책 너무 재미있어요! 추천합니다.,긍정,0.808976
3,별로인 영화. 보지 않는 것을 추천합니다.,부정,0.879885


## 요기요 리뷰 긍부정 평가

In [ ]:
train

,review,owner_reply
0,오늘도맛있게해주셔서감사합니다,"m0님,안녕하세요 시간내어 리뷰작성해주셔서 정말 감사합니다항상 고객님들 주문 하나하..."
1,너무 맛있어요 감사합니다,저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다계속해서 더 많은 기대 부탁드...
2,먹다가 정신이 없어서 2조각 순삭하고 찍었네요,식사는 맛있게 하셨나요? 혜화 대학로 SNS피자랭킹맛집피자보이시나 대학로점을 믿고주...
3,맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요,만족해주신 그 맛! 절대 변하지 않고 앞으로도 쭉 만족하실 수 있도록 최선을 다하겠...
4,주말저녁 맛있게먹었어요,안녕하세요 고객님 D번거로우실텐데 리뷰 작성해 주셔서 감사합니다!앞으로도 맛있게 ...
...,...,...
15195,맛도 너무 좋고 양도 넉넉하고 시간은 겁나 빠르게 맛있습니다. 재주문 나중...,맛있게 드셔주셔서 감사드립니다!!맛도 좋고 양도 좋고 가성비좋고3박자가 다 어우러진...
15196,혼자먹기 많아용 배달은 20분만에 왔어요 잘먹었습니다,황궁 찾아주셔서 넘넘 감사합니다세스코사용으로 청결하고 빠른배달과 친절한서비스로찾아뵙...
15197,항상 맛있게 먹고 있습니다.,안녕하세요.삽교원조두리곱창입니다.저희 음식이 입맛에 맞으셨다니 정말 다행이고 뿌듯합...
15198,맛있어서 먹다가 중간에 찍었어요 3인이서 폭풍흡입중 맛있네요,고돼지 송파점입니다 별점만점 그리고 따뜻한 리뷰 작성해주셔서 감사합니다.위생과 청결...


In [ ]:
len(train['review'].tolist()), len(train['owner_reply'].tolist())

(15200, 15200)

In [ ]:
reviews = train['review'].tolist()
reviews

['오늘도맛있게해주셔서감사합니다',
 '너무 맛있어요 감사합니다 ',
 '먹다가 정신이 없어서 2조각 순삭하고 찍었네요 ',
 '맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요',
 '주말저녁 맛있게먹었어요',
 '좋았습니다 김치 더 주시면 감사하겠습니다',
 '진짜 너무 맛있어요 여기에서만 시켜용',
 '첫 주문이였는데 기대 이상입니다! 덕분에 오랜만애 기분좋은 시간 보냈습니다오자마자 허겁지겁먹어서 사진은 깜빡했네요 종종 또 찾아 주문 하겠습니다!!',
 '다리살이라 맛잇어욥양념도 자랑할만',
 '맛있게 먹었습니다. 양이 정말 많네요 다음에 또 시킬 거 같아요',
 '이틀 연속 시킬정도로 존맛',
 '저번 정식도 맛있었는데 가자미 미역국도 맛있네요비린내 걱정을 했는데 전혀 안나고 시원 담백합니다몸 안좋거나 해장할때 너무 좋을듯',
 '토핑도 풍부하고 맛있게 잘먹었습니다',
 '너무 맛있었고요 추가주문한게 빠졌다고 연락드렸더니 서비스와 함께 바로 보내주셨어요자주 시켜먹게 될 것 같아요',
 '와플은  바삭바삭맛있어요떡볶이도 괜찮긴한데...국물과떡이 조금 따로 노는 기분이에요..  떡이 그래서 흰색에 가까웠어요  맛 자체는 괜찮은 편입니다많이파세요',
 '맛있게 잘 먹었습니다',
 '맛있습니다 양도 괜찮습니다',
 '피자위에 콜라를 올려두셨는지모르겠고한데젖어있고 피자가 좀 찌그러져있고치즈가 이미 식어서 굳어있네요좀 아쉽고 한데피자는 치즈가 늘어나는맛에 먹는건데다음엔 따듯한걸먹길...',
 '리뷰가 늦었네요 너무 맛있게 먹었습니다!!진짜 존맛!!',
 '맛있어요......',
 '육회비빔밥은 여기서만 시킵니다 진짜 너무맛있어요 집근처 지점말고 본점에서만 계속주문합니다 번창하세요',
 '피자는 피보시 배달도 빠르고 서비스도 짱이고 뭣보다 맛있음',
 '그릇이 특이해요 사이드메뉴가 맛있어요 새우링 체고',
 '리뷰늦었지만  진짜 맛있게잘먹었습니다  요즘 피자여기서밖에안먹어요',
 '처음으로 훈제치킨 조합으로 먹었는데 역시 실망없는 맛이였습니다',
 '꽁치도 맛

- 30개

### 확인ver : random 100개

In [ ]:
# 리뷰 분석 및 데이터프레임 반환
result_df = analyze_reviews(reviews[:100], model_name)
pd.DataFrame(result_df)

,review,sentiment,score
0,오늘도맛있게해주셔서감사합니다,긍정,0.785271
1,너무 맛있어요 감사합니다,긍정,0.873893
2,먹다가 정신이 없어서 2조각 순삭하고 찍었네요,긍정,0.585508
3,맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요,긍정,0.806666
4,주말저녁 맛있게먹었어요,긍정,0.493130
...,...,...,...
95,밑반찬도맛있고 오징어도 많이들었습니다 좋아요,긍정,0.615110
96,너무 맛있게 잘먹었어요 하 감동스러운 맛입니다,긍정,0.895166
97,맛있고 양이 많아요,긍정,0.508591
98,잘 먹었습니다 술안주로도 좋아요,긍정,0.485113


- 긍정 리뷰 : 90%
- 대체로 정확도 높음

In [ ]:
mask = result_df['sentiment'] == '긍정'
positive_reviews = result_df[mask]
positive_reviews


,review,sentiment,score
0,오늘도맛있게해주셔서감사합니다,긍정,0.785271
1,너무 맛있어요 감사합니다,긍정,0.873893
2,먹다가 정신이 없어서 2조각 순삭하고 찍었네요,긍정,0.585508
3,맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요,긍정,0.806666
4,주말저녁 맛있게먹었어요,긍정,0.493130
...,...,...,...
95,밑반찬도맛있고 오징어도 많이들었습니다 좋아요,긍정,0.615110
96,너무 맛있게 잘먹었어요 하 감동스러운 맛입니다,긍정,0.895166
97,맛있고 양이 많아요,긍정,0.508591
98,잘 먹었습니다 술안주로도 좋아요,긍정,0.485113


In [ ]:
mask = (positive_reviews['score'] > 0.90) | (positive_reviews['score'] < 0.10)
positive_reviews[mask]

,review,sentiment,score
46,포장 주문 했는데 준비 시간도 딱 맞춰 주시고 요청사항도 잘 들어 주셨어요. 덕분에...,긍정,0.925508
52,정말 말이 필요없는 최고의 피자맛집...,긍정,0.915255


- 부정 리뷰 : 10%

In [ ]:
mask = result_df['sentiment'] == '부정'
negative_reviews = result_df[mask]
negative_reviews

,review,sentiment,score
14,와플은 바삭바삭맛있어요떡볶이도 괜찮긴한데...국물과떡이 조금 따로 노는 기분이에요...,부정,0.558067
17,피자위에 콜라를 올려두셨는지모르겠고한데젖어있고 피자가 좀 찌그러져있고치즈가 이미 식...,부정,0.515175
31,맛있게 먹고 있었는데 머리카락 발견,부정,0.770465
49,비오는날이라 배달 늦는건 이해한다고 해도..버팔로봉이 비닐에 나뒹굴어서 기분나빠서 ...,부정,0.592872
57,처음배달온 사진..음식을 위아래로 흔들어왔나보네요씬도우가 도우만 씬이 아니고 모든 ...,부정,0.997797
67,?????리뷰......식혜........콘치즈구이....한강.....왜....그런...,부정,0.794605
76,적은거랑.다른리뷰이벤트음식이 왔지만 전이 너무맛있어서 상관이없네요,부정,0.385860
82,마라샹궈 치고 생각보다 싱거웠어용그래두 맛있긴함 밥 양 짱많음,부정,0.411574


In [ ]:
positive_reviews.shape, negative_reviews.shape

((92, 3), (8, 3))

### 전체ver : 15000개
- 리뷰가 대용량인 경우 위 간단 평가 함수 오류남

- 데이터셋 정의하기

In [ ]:
class ReviewDataset(review_list):
  self.

In [ ]:
import torch
import pandas as pd
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

def analyze_reviews(review_list, model_name, batch_size=2):
    model = GPT2ForSequenceClassification.from_pretrained(model_name)

    tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                      bos_token='</s>',
                                      eos_token='</s>',
                                      unk_token='<unk>',
                                      pad_token='<pad>',
                                      mask_token='<mask>',
                                      max_len=1024)

    responses = []
    for i in range(0, len(review_list), batch_size):
        batch_reviews = review_list[i:i+batch_size]

        inputs = tokenizer(batch_reviews, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True)

        with torch.no_grad():
            result = model(**inputs)

        logits = result.logits
        probs = torch.softmax(logits, dim=-1)
        pos_probs = probs[:, 1]  # 긍정 클래스의 확률 값

        for j, pos_prob in enumerate(pos_probs):
            sentiment = "긍정" if pos_prob > 0.5 else "부정"
            responses.append((sentiment, pos_prob.item()))

    return responses


In [ ]:

# 모델 이름 설정
model_name = "MLOpsEngineer/review-rate-prediction"


# 리뷰 분석 실행
result_responses = analyze_reviews(reviews, model_name, batch_size=2)

# 결과를 데이터프레임으로 변환하여 출력
result_df = pd.DataFrame({'리뷰': reviews, '감정': [res[0] for res in result_responses], '긍정 확률': [res[1] for res in result_responses]})
print(result_df)


You are using a model of type bert to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at MLOpsEngineer/review-rate-prediction and are newly initialized: ['h.9.attn.c_proj.bias', 'h.4.ln_2.bias', 'h.5.ln_1.weight', 'h.2.attn.c_attn.weight', 'h.0.attn.c_attn.weight', 'h.11.mlp.c_proj.weight', 'h.4.ln_1.weight', 'h.10.attn.c_proj.bias', 'h.8.mlp.c_fc.bias', 'h.8.ln_2.weight', 'h.8.mlp.c_fc.weight', 'h.2.ln_1.bias', 'h.4.attn.c_attn.weight', 'h.6.mlp.c_proj.bias', 'h.9.mlp.c_proj.weight', 'h.4.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.9.ln_2.bias', 'h.4.attn.c_attn.bias', 'h.9.ln_1.weight', 'h.0.attn.c_attn.bias', 'h.10.ln_2.weight', 'score.weight', 'h.0.ln_2.weight', 'h.1.ln_2.weight', 'h.7.attn.c_proj.bias', 'h.11.attn.c_attn.weight', 'h.4.mlp.c_fc.weight', 'h.5.ln_2.weight', 'h.4.mlp.c_proj.weight', 'h.3.mlp.c_fc.bias', 'h.3.attn.c

IndexError: ignored

In [ ]:
# 리뷰 분석 및 데이터프레임 반환

result_df = analyze_reviews(reviews, model_name)
pd.DataFrame(result_df)

ValueError: ignored

In [ ]:
result_df = analyze_reviews(reviews, model_name)
pd.DataFrame(result_df)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: ignored

In [ ]:
reviews

['이 영화 정말 좋았습니다. 감동적인 내용과 연기가 인상 깊었어요.',
 '아쉬운 영화였습니다. 전반적으로 기대에 못 미쳤습니다.',
 '이 책 너무 재미있어요! 추천합니다.',
 '별로인 영화. 보지 않는 것을 추천합니다.']

In [ ]:
# import torch
# import pandas as pd
# from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

# def analyze_reviews(review_list, model_name):

#     model = GPT2ForSequenceClassification.from_pretrained(model_name)
#     # tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#     tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
#                                           bos_token='</s>',
#                                           eos_token='</s>',
#                                           unk_token='<unk>',
#                                           pad_token='<pad>',
#                                           mask_token='<mask>',
#                                           max_len=1024)


#     responses = []
#     for review in review_list:
#         # 토크나이저 입력 형식에 맞게 텍스트 가공
#         input_text = f"리뷰: {review}"

#         # 토큰화 및 인코딩
#         inputs = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True)

#         # 모델 추론
#         with torch.no_grad():
#             result = model(inputs)

#         # 결과를 확률값으로 변환하여 긍정 부정 판단
#         logits = result.logits
#         probs = torch.softmax(logits, dim=-1)
#         pos_prob = probs[0][1].item()  # 긍정 클래스의 확률 값

#         # 확률 값을 기반으로 긍정 부정 판단
#         sentiment = "긍정" if pos_prob > 0.5 else "부정"

#         responses.append((sentiment, pos_prob))

#     return responses


You are using a model of type bert to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at MLOpsEngineer/review-rate-prediction and are newly initialized: ['h.9.attn.c_proj.bias', 'h.4.ln_2.bias', 'h.5.ln_1.weight', 'h.2.attn.c_attn.weight', 'h.0.attn.c_attn.weight', 'h.11.mlp.c_proj.weight', 'h.4.ln_1.weight', 'h.10.attn.c_proj.bias', 'h.8.mlp.c_fc.bias', 'h.8.ln_2.weight', 'h.8.mlp.c_fc.weight', 'h.2.ln_1.bias', 'h.4.attn.c_attn.weight', 'h.6.mlp.c_proj.bias', 'h.9.mlp.c_proj.weight', 'h.4.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.9.ln_2.bias', 'h.4.attn.c_attn.bias', 'h.9.ln_1.weight', 'h.0.attn.c_attn.bias', 'h.10.ln_2.weight', 'score.weight', 'h.0.ln_2.weight', 'h.1.ln_2.weight', 'h.7.attn.c_proj.bias', 'h.11.attn.c_attn.weight', 'h.4.mlp.c_fc.weight', 'h.5.ln_2.weight', 'h.4.mlp.c_proj.weight', 'h.3.mlp.c_fc.bias', 'h.3.attn.c

                                     리뷰  감정     긍정 확률
0  이 영화 정말 좋았습니다. 감동적인 내용과 연기가 인상 깊었어요.  부정  0.119108
1        아쉬운 영화였습니다. 전반적으로 기대에 못 미쳤습니다.  부정  0.201774
2                  이 책 너무 재미있어요! 추천합니다.  부정  0.114193
3               별로인 영화. 보지 않는 것을 추천합니다.  부정  0.100771


In [ ]:

# # 분류할 리뷰 리스트
# reviews = [
#     "이 영화 정말 좋았습니다. 감동적인 내용과 연기가 인상 깊었어요.",
#     "아쉬운 영화였습니다. 전반적으로 기대에 못 미쳤습니다.",
#     "이 책 너무 재미있어요! 추천합니다.",
#     "별로인 영화. 보지 않는 것을 추천합니다."
# ]

# # 모델 이름 설정
# model_name = "MLOpsEngineer/review-rate-prediction"

# # 리뷰 분석 실행
# result_responses = analyze_reviews(reviews, model_name)

# # 결과를 데이터프레임으로 변환하여 출력
# result_df = pd.DataFrame({'리뷰': reviews, '감정': [res[0] for res in result_responses], '긍정 확률': [res[1] for res in result_responses]})
# print(result_df)


You are using a model of type bert to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at MLOpsEngineer/review-rate-prediction and are newly initialized: ['h.9.attn.c_proj.bias', 'h.4.ln_2.bias', 'h.5.ln_1.weight', 'h.2.attn.c_attn.weight', 'h.0.attn.c_attn.weight', 'h.11.mlp.c_proj.weight', 'h.4.ln_1.weight', 'h.10.attn.c_proj.bias', 'h.8.mlp.c_fc.bias', 'h.8.ln_2.weight', 'h.8.mlp.c_fc.weight', 'h.2.ln_1.bias', 'h.4.attn.c_attn.weight', 'h.6.mlp.c_proj.bias', 'h.9.mlp.c_proj.weight', 'h.4.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.9.ln_2.bias', 'h.4.attn.c_attn.bias', 'h.9.ln_1.weight', 'h.0.attn.c_attn.bias', 'h.10.ln_2.weight', 'score.weight', 'h.0.ln_2.weight', 'h.1.ln_2.weight', 'h.7.attn.c_proj.bias', 'h.11.attn.c_attn.weight', 'h.4.mlp.c_fc.weight', 'h.5.ln_2.weight', 'h.4.mlp.c_proj.weight', 'h.3.mlp.c_fc.bias', 'h.3.attn.c

                                     리뷰  감정     긍정 확률
0  이 영화 정말 좋았습니다. 감동적인 내용과 연기가 인상 깊었어요.  부정  0.135719
1        아쉬운 영화였습니다. 전반적으로 기대에 못 미쳤습니다.  부정  0.068459
2                  이 책 너무 재미있어요! 추천합니다.  부정  0.110079
3               별로인 영화. 보지 않는 것을 추천합니다.  부정  0.069024


# ------- 참고 --------------------

## 참고.PyKoSpacing (띄어쓰기)

- https://github.com/haven-jeon/PyKoSpacing

In [ ]:
!git clone https://github.com/haven-jeon/PyKoSpacing.git

fatal: destination path 'PyKoSpacing' already exists and is not an empty directory.


In [ ]:
!cd PyKoSpacing


In [ ]:
# !python3 setup.py install


python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [ ]:
!pip install PyKoSpacing/

Processing ./PyKoSpacing
  Preparing metadata (setup.py) ... done
  Using cached tensorflow-2.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.8 MB)
  Using cached h5py-3.1.0.tar.gz (371 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install backend dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
from PyKoSpacing import Spacing

spacing = Spacing()

# 예시
spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")

spacing = Spacing(rules=['구레나룻'])
spacing('귀밑에서턱까지잇따라난수염을구레나룻이라고한다.')